<a href="https://colab.research.google.com/github/jpanda001/Artistic-Style-Transfer/blob/master/Style_Transfer_Model_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.models as models

In [0]:
vgg19 = models.vgg19(pretrained=True).features.eval()

In [6]:
from torchsummary import summary
summary(vgg19, input_size=(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
         MaxPool2d-5         [-1, 64, 128, 128]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 128, 128]         147,584
              ReLU-9        [-1, 128, 128, 128]               0
        MaxPool2d-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]         295,168
             ReLU-12          [-1, 256, 64, 64]               0
           Conv2d-13          [-1, 256, 64, 64]         590,080
             ReLU-14          [-1, 256,

In [0]:
## I dont think we need this at all
class Vgg(nn.Module): #
  def __init__(self,num_hidden):
    super(Vgg,self).__init__()                    
    self.name = "Vgg"
    self.fc1 = nn.Linear(1000,100)  
    self.fc2 = nn.Linear(100,10)
    self.fc3 = nn.Linear(10,2)       #output size = ?                              
    self.num_hidden = num_hidden

  def forward(self, x):
    x = F.relu(self.fc1(x))                               #apply relu activation function to make sure it's not linear relationship, output=100                         
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc(x)) 
    return x

In [0]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = nn.Sequential( 
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [0]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
#conv layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=9, stride=1)      #How to implement batch normalization?
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, stride=1)
        self.pool = nn.maxpool2d(2,2)
# Res layers
  
  #How to build residual Layers?

# deconv Layers
        self.de_conv1 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1)
        self.de_conv2 = nn.ConvTranspose2d(64, 32, kernel_size=5, stride=1)
        self.de_conv3 = nn.ConvTranspose2d(32, 3, kernel_size=5, stride=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
      # x = self.pool(x)
        x = F.relu(self.conv2(x))
      #  x = self.pool(x)
        x = F.relu(self.conv3(x))
      #  x = self.pool(x)  #28*28
      # x = self.res(x)  #?
        x = F.relu(self.de_conv1(x))
        x = F.relu(self.de_conv2(x))
        x = F.de_conv3(x)
        return x


In [0]:
# Training Network
def train(generator, discriminator, lr=0.001, num_epochs=5, style_weight = 100, content_weight = 1):
    criterion = nn.BCEWithLogitsLoss()
    #d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

    train_loader = torch.utils.data.DataLoader(mnist_data, batch_size=100, shuffle=True)

    num_test_samples = 16
    test_noise = torch.randn(num_test_samples, 100)

    content_layers = ['conv_4']
    style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    for epoch in range(num_epochs):
        # label that we are using both models 
        generator.train()
        discriminator.train()


        for n, (images, _) in enumerate(train_loader):
          
            for layers in vgg19.features:
              if isinstance(layers,conv_2d):
                i += 1
                name = 'conv_{}'.format(i)
                            
              if name in content_layers:
                noise = torch.randn(images.size(0), 100)
                transformed_images = generator(noise)
                transformed_content = vgg19(transformed_images)
                content_content = vgg19(content_image)
                content_loss = F.mse_loss(transformed_content, content_content)
              
              if name in style_layers:
                noise = torch.randn(images.size(0), 100)
                transformed_images = generator(noise)
                transformed_style = vgg19(transformed_images)
                style_style = vgg19(style_image)
                style_loss = F.mse_loss(transformed_style, style_style)
              
              g_loss = style_weight*style_loss + content_weight*content_loss
              g_loss.backward()
              g_optimizer.step()
            